使用下面的命令打开端口转发，然后在本地使用服务器内核运行jupyter
```
ssh -L 8888:localhost:8888 zjb@`hiServer -pIP`
```

In [43]:
from fastai.vision.all import *
import torch

print(torch.cuda.is_available())
torch.cuda.set_device(3)

True


In [44]:
#hide
# ! [ -e /content ] && pip install -Uqq fastbook
import fastbook
fastbook.setup_book()

In [45]:
#hide
print("test")
from fastbook import *
from IPython.display import display,HTML

test


# NLP Deep Dive: RNNs

In <<chapter_intro>> we saw that deep learning can be used to get great results with natural language datasets. Our example relied on using a pretrained language model and fine-tuning it to classify reviews. That example highlighted a difference between transfer learning in NLP and computer vision: in general in NLP the pretrained model is trained on a different task.

What we call a language model is a model that has been trained to guess what the next word in a text is (having read the ones before). This kind of task is called *self-supervised learning*: we do not need to give labels to our model, just feed it lots and lots of texts. It has a process to automatically get labels from the data, and this task isn't trivial: to properly guess the next word in a sentence, the model will have to develop an understanding of the English (or other) language. Self-supervised learning can also be used in other domains; for instance, see ["Self-Supervised Learning and Computer Vision"](https://www.fast.ai/2020/01/13/self_supervised/) for an introduction to vision applications. Self-supervised learning is not usually used for the model that is trained directly, but instead is used for pretraining a model used for transfer learning.

在<<chapter_intro>>中，我们看到深度学习可以通过自然语言数据集获得很好的结果。我们的示例依赖于使用预训练的语言模型并对其进行微调以对评论进行分类。这个例子强调了 NLP 和计算机视觉中的迁移学习之间的区别：一般来说，在 NLP 中，预训练模型是针对不同的任务进行训练的。

我们所说的语言模型是一种经过训练可以猜测文本中的下一个单词是什么（之前读过的单词）的模型。这种任务称为`自我监督学习`：我们不需要给模型添加标签，只需向其提供大量文本即可。它有一个自动从数据中获取标签的过程，而这项任务并不简单：为了正确猜测句子中的下一个单词，模型必须培养对英语（或其他）语言的理解。自监督学习也可以用于其他领域；例如，有关视觉应用的介绍，请参阅[`自监督学习和计算机视觉`](https://www.fast.ai/2020/01/13/self_supervised/)。自监督学习通常不用于直接训练的模型，而是用于预训练用于迁移学习的模型。

> jargon: Self-supervised learning: Training a model using labels that are embedded in the independent variable, rather than requiring external labels. For instance, training a model to predict the next word in a text.

> 行话：自监督学习：使用嵌入自变量中的标签来训练模型，而不需要外部标签。例如，训练模型来预测文本中的下一个单词。

The language model we used in <<chapter_intro>> to classify IMDb reviews was pretrained on Wikipedia. We got great results by directly fine-tuning this language model to a movie review classifier, but with one extra step, we can do even better. The Wikipedia English is slightly different from the IMDb English, so instead of jumping directly to the classifier, we could fine-tune our pretrained language model to the IMDb corpus and then use *that* as the base for our classifier.

Even if our language model knows the basics of the language we are using in the task (e.g., our pretrained model is in English), it helps to get used to the style of the corpus we are targeting. It may be more informal language, or more technical, with new words to learn or different ways of composing sentences. In the case of the IMDb dataset, there will be lots of names of movie directors and actors, and often a less formal style of language than that seen in Wikipedia.

We already saw that with fastai, we can download a pretrained English language model and use it to get state-of-the-art results for NLP classification. (We expect pretrained models in many more languages to be available soon—they might well be available by the time you are reading this book, in fact.) So, why are we learning how to train a language model in detail?

One reason, of course, is that it is helpful to understand the foundations of the models that you are using. But there is another very practical reason, which is that you get even better results if you fine-tune the (sequence-based) language model prior to fine-tuning the classification model. For instance, for the IMDb sentiment analysis task, the dataset includes 50,000 additional movie reviews that do not have any positive or negative labels attached. Since there are 25,000 labeled reviews in the training set and 25,000 in the validation set, that makes 100,000 movie reviews altogether. We can use all of these reviews to fine-tune the pretrained language model, which was trained only on Wikipedia articles; this will result in a language model that is particularly good at predicting the next word of a movie review.

This is known as the Universal Language Model Fine-tuning (ULMFit) approach. The [paper](https://arxiv.org/abs/1801.06146) showed that this extra stage of fine-tuning of the language model, prior to transfer learning to a classification task, resulted in significantly better predictions. Using this approach, we have three stages for transfer learning in NLP, as summarized in <<ulmfit_process>>.

我们在 <<chapter_intro>> 中用于对 IMDb 评论进行分类的语言模型是在维基百科上进行预训练的。通过直接将此语言模型微调为电影评论分类器，我们获得了很好的结果，但通过一个额外的步骤，我们可以做得更好。维基百科英语与 IMDb 英语略有不同，因此我们可以将预训练的语言模型微调到 IMDb 语料库，然后使用 *that* 作为我们分类器的基础，而不是直接跳转到分类器。

即使我们的语言模型知道我们在任务中使用的语言的基础知识（例如，我们的预训练模型是英语），它也有助于适应我们目标语料库的风格。它可能是更非正式的语言，或更技术性的语言，需要学习新单词或不同的造句方式。就 IMDb 数据集而言，会有大量电影导演和演员的名字，并且通常采用比维基百科中不太正式的语言风格。

我们已经看到，通过 fastai，我们可以下载预训练的英语语言模型，并使用它来获得 NLP 分类的最新结果。 （我们预计更多语言的预训练模型很快就会推出——事实上，当你阅读本书时，它们很可能已经可用。）那么，我们为什么要学习如何详细训练语言模型呢？

当然，原因之一是了解您正在使用的模型的基础很有帮助。但还有另一个非常实际的原因，那就是如果在微调分类模型之前微调（基于序列的）语言模型，您会得到更好的结果。例如，对于 IMDb 情感分析任务，数据集包含 50,000 条额外的电影评论，这些评论没有附加任何正面或负面标签。由于训练集中有 25,000 条带标签的评论，验证集中有 25,000 条带标签的评论，因此总共有 100,000 条电影评论。我们可以使用所有这些评论来微调预训练的语言模型，该模型仅在维基百科文章上进行训练；这将产生一个特别擅长预测电影评论的下一个单词的语言模型。

这称为通用语言模型微调 (ULMFit) 方法。 [论文](https://arxiv.org/abs/1801.06146) 表明，在将学习转移到分类任务之前，对语言模型进行额外的微调可以产生更好的预测。使用这种方法，我们在 NLP 中进行迁移学习的三个阶段，如《ulmfit_process》中所总结的那样。

<img alt="Diagram of the ULMFiT process" width="700" caption="The ULMFiT process" id="ulmfit_process" src="images/att_00027.png">

We'll now explore how to apply a neural network to this language modeling problem, using the concepts introduced in the last two chapters. But before reading further, pause and think about how *you* would approach this.

现在，我们将使用前两章中介绍的概念，探索如何将神经网络应用于此语言建模问题。但在进一步阅读之前，请暂停并思考`您`将如何解决这个问题。

## Text Preprocessing

## 文本预处理

It's not at all obvious how we're going to use what we've learned so far to build a language model. Sentences can be different lengths, and documents can be very long. So, how can we predict the next word of a sentence using a neural network? Let's find out!

We've already seen how categorical variables can be used as independent variables for a neural network. The approach we took for a single categorical variable was to:

1. Make a list of all possible levels of that categorical variable (we'll call this list the *vocab*).
1. Replace each level with its index in the vocab.
1. Create an embedding matrix for this containing a row for each level (i.e., for each item of the vocab).
1. Use this embedding matrix as the first layer of a neural network. (A dedicated embedding matrix can take as inputs the raw vocab indexes created in step 2; this is equivalent to but faster and more efficient than a matrix that takes as input one-hot-encoded vectors representing the indexes.)

We can do nearly the same thing with text! What is new is the idea of a sequence. First we concatenate all of the documents in our dataset into one big long string and split it into words, giving us a very long list of words (or "tokens"). Our independent variable will be the sequence of words starting with the first word in our very long list and ending with the second to last, and our dependent variable will be the sequence of words starting with the second word and ending with the last word. 

Our vocab will consist of a mix of common words that are already in the vocabulary of our pretrained model and new words specific to our corpus (cinematographic terms or actors names, for instance). Our embedding matrix will be built accordingly: for words that are in the vocabulary of our pretrained model, we will take the corresponding row in the embedding matrix of the pretrained model; but for new words we won't have anything, so we will just initialize the corresponding row with a random vector.

我们将如何使用迄今为止所学到的知识来构建语言模型并不明显。句子的长度可以不同，文档也可以很长。那么，我们如何使用神经网络预测句子的下一个单词呢？让我们来看看吧！

我们已经了解了如何将分类变量用作神经网络的自变量。我们对单个分类变量采取的方法是：

1. 列出该分类变量的所有可能级别（我们将此列表称为 *vocab*）。
1. 将每个级别替换为其在词汇表中的索引。
1. 为此创建一个嵌入矩阵，其中包含每个级别（即词汇的每个项目）的一行。
1. 使用该嵌入矩阵作为神经网络的第一层。 （专用嵌入矩阵可以将步骤 2 中创建的原始词汇索引作为输入；这与将表示索引的 one-hot 编码向量作为输入的矩阵等效，但更快、更高效。）

我们可以用文本做几乎同样的事情！新的是序列的想法。首先，我们将数据集中的所有文档连接成一个大的长字符串，并将其拆分为单词，从而为我们提供一个非常长的单词列表（或`标记`）。我们的自变量将是从很长的列表中的第一个单词开始到倒数第二个单词结束的单词序列，我们的因变量将是从第二个单词开始到最后一个单词结束的单词序列。 

我们的词汇将由预训练模型词汇表中已有的常用单词和语料库特有的新单词（例如电影术语或演员姓名）组成。我们的嵌入矩阵将相应地构建：对于预训练模型词汇表中的单词，我们将采用预训练模型嵌入矩阵中的相应行；但对于新单词，我们不会有任何东西，因此我们将用随机向量初始化相应的行。

Each of the steps necessary to create a language model has jargon associated with it from the world of natural language processing, and fastai and PyTorch classes available to help. The steps are:

- Tokenization:: Convert the text into a list of words (or characters, or substrings, depending on the granularity of your model)
- Numericalization:: Make a list of all of the unique words that appear (the vocab), and convert each word into a number, by looking up its index in the vocab
- Language model data loader creation:: fastai provides an `LMDataLoader` class which automatically handles creating a dependent variable that is offset from the independent variable by one token. It also handles some important details, such as how to shuffle the training data in such a way that the dependent and independent variables maintain their structure as required
- Language model creation:: We need a special kind of model that does something we haven't seen before: handles input lists which could be arbitrarily big or small. There are a number of ways to do this; in this chapter we will be using a *recurrent neural network* (RNN). We will get to the details of these RNNs in the <<chapter_nlp_dive>>, but for now, you can think of it as just another deep neural network.

Let's take a look at how each step works in detail.

创建语言模型所需的每个步骤都有来自自然语言处理领域的与之相关的术语，并且 fastai 和 PyTorch 类可提供帮助。步骤是：

- 标记化：：将文本转换为单词列表（或字符或子字符串，具体取决于模型的粒度）
- 数字化：列出所有出现的唯一单词（词汇），并通过在词汇中查找其索引将每个单词转换为数字
- 语言模型数据加载器创建：fastai 提供了一个 `LMDataLoader` 类，它自动处理创建一个从自变量偏移一个标记的因变量。它还处理一些重要的细节，例如如何以因变量和自变量根据需要保持其结构的方式对训练数据进行洗牌
- 语言模型创建：：我们需要一种特殊的模型来完成我们以前从未见过的事情：处理可以任意大或任意小的输入列表。有多种方法可以做到这一点；在本章中，我们将使用*循环神经网络*（RNN）。我们将在 <<chapter_nlp_dive>> 中详细了解这些 RNN，但现在，您可以将其视为另一个深度神经网络。

让我们看一下每个步骤的详细工作原理。

### Tokenization

### 转化为Token

When we said "convert the text into a list of words," we left out a lot of details. For instance, what do we do with punctuation? How do we deal with a word like "don't"? Is it one word, or two? What about long medical or chemical words? Should they be split into their separate pieces of meaning? How about hyphenated words? What about languages like German and Polish where we can create really long words from many, many pieces? What about languages like Japanese and Chinese that don't use bases at all, and don't really have a well-defined idea of *word*?

Because there is no one correct answer to these questions, there is no one approach to tokenization. There are three main approaches:

- Word-based:: Split a sentence on spaces, as well as applying language-specific rules to try to separate parts of meaning even when there are no spaces (such as turning "don't" into "do n't"). Generally, punctuation marks are also split into separate tokens.
- Subword based:: Split words into smaller parts, based on the most commonly occurring substrings. For instance, "occasion" might be tokenized as "o c ca sion."
- Character-based:: Split a sentence into its individual characters.

We'll be looking at word and subword tokenization here, and we'll leave character-based tokenization for you to implement in the questionnaire at the end of this chapter.

当我们说`将文本转换为单词列表`时，我们遗漏了很多细节。例如，我们如何处理标点符号？我们如何处理`不`这样的词？是一个字还是两个字？长的医学或化学单词怎么样？它们是否应该被分成各自独立的含义？带连字符的单词怎么样？像德语和波兰语这样的语言怎么样，我们可以从很多很多的片段中创造出非常长的单词？像日语和中文这样完全不使用基数并且实际上没有明确定义的*单词*概念的语言又如何呢？

因为这些问题没有一个正确的答案，所以也没有一种标记化方法。主要有以下三种方法：

- 基于单词：：在空格上分割句子，以及应用特定于语言的规则来尝试分离含义的各个部分，即使没有空格（例如将`don't`变成`do n't`） 。一般来说，标点符号也被分成单独的标记。
- 基于子词：：根据最常出现的子字符串将单词分割成更小的部分。例如，`occasion`可能被标记为`oc ca sion`。
- 基于字符：：将句子拆分为各个字符。

我们将在这里讨论单词和子词标记化，并且我们将把基于字符的标记化留给您在本章末尾的调查问卷中实施。

> jargon: token: One element of a list created by the tokenization process. It could be a word, part of a word (a _subword_), or a single character.

> 行话： token：标记化过程创建的列表的一个元素。它可以是一个单词、单词的一部分（_子单词_）或单个字符。

### Word Tokenization with fastai

### 使用 fastai 进行单词Tokenization

Rather than providing its own tokenizers, fastai instead provides a consistent interface to a range of tokenizers in external libraries. Tokenization is an active field of research, and new and improved tokenizers are coming out all the time, so the defaults that fastai uses change too. However, the API and options shouldn't change too much, since fastai tries to maintain a consistent API even as the underlying technology changes.

Let's try it out with the IMDb dataset that we used in <<chapter_intro>>:

fastai 没有提供自己的分词器，而是为外部库中的一系列分词器提供了一致的接口。标记化是一个活跃的研究领域，新的和改进的标记器不断出现，因此 fastai 使用的默认值也会发生变化。然而，API 和选项不应该改变太多，因为即使底层技术发生变化，fastai 也会尝试保持一致的 API。

让我们用 <<chapter_intro>> 中使用的 IMDb 数据集来尝试一下：

In [46]:
from fastai.text.all import *
path = untar_data(URLs.IMDB)

We'll need to grab the text files in order to try out a tokenizer. Just like `get_image_files`, which we've used many times already, gets all the image files in a path, `get_text_files` gets all the text files in a path. We can also optionally pass `folders` to restrict the search to a particular list of subfolders:

我们需要获取文本文件才能尝试分词器。就像我们已经使用过很多次的 `get_image_files` 获取路径中的所有图像文件一样，`get_text_files` 获取路径中的所有文本文件。我们还可以选择传递`folders`来将搜索限制为特定的子文件夹列表：

In [47]:
files = get_text_files(path, folders = ['train', 'test', 'unsup'])

Here's a review that we'll tokenize (we'll just print the start of it here to save space):

这是我们将tokenize的预览（我们将在此处打印它的开头以节省空间）：

In [48]:
txt = files[0].open().read()
print(txt[:75])
print(txt[1])

The racism in this disgraceful production amidst breathtaking scenery follo
h


As we write this book, the default English word tokenizer for fastai uses a library called *spaCy*. It has a sophisticated rules engine with special rules for URLs, individual special English words, and much more. Rather than directly using `SpacyTokenizer`, however, we'll use `WordTokenizer`, since that will always point to fastai's current default word tokenizer (which may not necessarily be spaCy, depending when you're reading this).

Let's try it out. We'll use fastai's `coll_repr(collection, n)` function to display the results. This displays the first *`n`* items of *`collection`*, along with the full size—it's what `L` uses by default. Note that fastai's tokenizers take a collection of documents to tokenize, so we have to wrap `txt` in a list:

在我们撰写本书时，fastai 的默认英语单词分词器使用一个名为 *spaCy* 的库。它有一个复杂的规则引擎，其中包含针对 URL、个别特殊英语单词等的特殊规则。然而，我们不会直接使用`SpacyTokenizer`，而是使用`WordTokenizer`，因为它将始终指向 fastai 当前的默认单词标记器（不一定是 spaCy，具体取决于您何时阅读本文）。

让我们尝试一下。我们将使用 fastai 的 coll_repr(collection, n) 函数来显示结果。这将显示 *`collection`* 的前 *`n`* 个项目，以及完整大小 - 这是 `L` 默认使用的内容。请注意，fastai 的分词器需要对文档集合进行分词，因此我们必须将`txt`包装在列表中：

In [49]:
spacy = WordTokenizer()
toks = first(spacy([txt]))
print(coll_repr(toks, 30))

(#144) ['The','racism','in','this','disgraceful','production','amidst','breathtaking','scenery','follows','the','pattern','of','similar','productions','from','its','source','and','time','.','We','see','hordes','of','Africans','slaughtering','innocent','whites','and'...]


As you see, spaCy has mainly just separated out the words and punctuation. But it does something else here too: it has split "it's" into "it" and "'s". That makes intuitive sense; these are separate words, really. Tokenization is a surprisingly subtle task, when you think about all the little details that have to be handled. Fortunately, spaCy handles these pretty well for us—for instance, here we see that "." is separated when it terminates a sentence, but not in an acronym or number:

正如你所看到的，spaCy 主要只是分离出了单词和标点符号。但它在这里也做了其他事情：它将`it's`分成了`it`和`'s`。这很直观。实际上，这些是单独的词。当您考虑必须处理的所有小细节时，标记化是一项令人惊讶的微妙任务。幸运的是，spaCy 为我们很好地处理了这些问题——例如，在这里我们看到了`.`。当它结束一个句子时被分隔，但不是首字母缩略词或数字：

In [50]:
first(spacy(['The U.S. dollar $1 is $1.00.']))

(#9) ['The','U.S.','dollar','$','1','is','$','1.00','.']

fastai then adds some additional functionality to the tokenization process with the `Tokenizer` class:

然后，fastai 使用`Tokenizer`类向标记化过程添加一些附加功能：

In [51]:
tkn = Tokenizer(spacy)
print(coll_repr(tkn(txt), 31))

(#161) ['xxbos','xxmaj','the','racism','in','this','disgraceful','production','amidst','breathtaking','scenery','follows','the','pattern','of','similar','productions','from','its','source','and','time','.','xxmaj','we','see','hordes','of','xxmaj','africans','slaughtering'...]


Notice that there are now some tokens that start with the characters "xx", which is not a common word prefix in English. These are *special tokens*.

For example, the first item in the list, `xxbos`, is a special token that indicates the start of a new text ("BOS" is a standard NLP acronym that means "beginning of stream"). By recognizing this start token, the model will be able to learn it needs to "forget" what was said previously and focus on upcoming words.

These special tokens don't come from spaCy directly. They are there because fastai adds them by default, by applying a number of rules when processing text. These rules are designed to make it easier for a model to recognize the important parts of a sentence. In a sense, we are translating the original English language sequence into a simplified tokenized language—a language that is designed to be easy for a model to learn.

For instance, the rules will replace a sequence of four exclamation points with a special *repeated character* token, followed by the number four, and then a single exclamation point. In this way, the model's embedding matrix can encode information about general concepts such as repeated punctuation rather than requiring a separate token for every number of repetitions of every punctuation mark. Similarly, a capitalized word will be replaced with a special capitalization token, followed by the lowercase version of the word. This way, the embedding matrix only needs the lowercase versions of the words, saving compute and memory resources, but can still learn the concept of capitalization.

Here are some of the main special tokens you'll see:

- `xxbos`:: Indicates the beginning of a text (here, a review)
- `xxmaj`:: Indicates the next word begins with a capital (since we lowercased everything)
- `xxunk`:: Indicates the word is unknown

To see the rules that were used, you can check the default rules:

请注意，现在有一些标记以字符`xx`开头，这在英语中不是常见的单词前缀。这些是*特殊令牌*。

例如，列表中的第一项`xxbos`是一个特殊标记，指示新文本的开始（`BOS`是标准 NLP 缩写，意思是`流的开始`）。通过识别这个开始标记，该模型将能够学习它需要`忘记`之前所说的内容并专注于接下来的单词。

这些特殊令牌并不直接来自 spaCy。它们之所以存在，是因为 fastai 在处理文本时应用了许多规则，默认添加它们。这些规则旨在使模型更容易识别句子的重要部分。从某种意义上说，我们正在将原始英语语言序列翻译成简化的标记语言——一种旨在使模型易于学习的语言。

例如，规则将用特殊的*repeat charactor*标记替换四个感叹号的序列，后跟数字4，然后是单个感叹号'!'。通过这种方式，模型的嵌入矩阵可以对有关一般概念（例如重复标点符号）的信息进行编码，而不需要为每个标点符号的每次重复次数使用单独的标记。同样，大写单词将被替换为特殊的大写标记，后跟该单词的小写版本。这样，嵌入矩阵只需要单词的小写版本，节省了计算和内存资源，但仍然可以学习大写的概念。

以下是您将看到的一些主要特殊标记：

- `xxbos`:: 表示文本的开头（这里是预览）
- `xxmaj`:: 表示下一个单词以大写开头（因为我们将所有内容都小写）
- `xxunk`:: 表示该词未知

要查看使用的规则，您可以检查默认规则：

In [52]:
defaults.text_proc_rules

[<function fastai.text.core.fix_html(x)>,
 <function fastai.text.core.replace_rep(t)>,
 <function fastai.text.core.replace_wrep(t)>,
 <function fastai.text.core.spec_add_spaces(t)>,
 <function fastai.text.core.rm_useless_spaces(t)>,
 <function fastai.text.core.replace_all_caps(t)>,
 <function fastai.text.core.replace_maj(t)>,
 <function fastai.text.core.lowercase(t, add_bos=True, add_eos=False)>]

As always, you can look at the source code of each of them in a notebook by typing:

```
??replace_rep
```

Here is a brief summary of what each does:

- `fix_html`:: Replaces special HTML characters with a readable version (IMDb reviews have quite a few of these)
- `replace_rep`:: Replaces any character repeated three times or more with a special token for repetition (`xxrep`), the number of times it's repeated, then the character
- `replace_wrep`:: Replaces any word repeated three times or more with a special token for word repetition (`xxwrep`), the number of times it's repeated, then the word
- `spec_add_spaces`:: Adds spaces around / and #
- `rm_useless_spaces`:: Removes all repetitions of the space character
- `replace_all_caps`:: Lowercases a word written in all caps and adds a special token for all caps (`xxup`) in front of it
- `replace_maj`:: Lowercases a capitalized word and adds a special token for capitalized (`xxmaj`) in front of it
- `lowercase`:: Lowercases all text and adds a special token at the beginning (`xxbos`) and/or the end (`xxeos`)

与往常一样，您可以通过键入以下内容在笔记本中查看它们每个的源代码：

````
??replace_rep
````

以下是每个功能的简要总结：

- `fix_html`:: 用可读版本替换特殊的 HTML 字符（IMDb 评论有很多这样的内容）
- `replace_rep`:: 用特殊的重复标记（`xxrep`）替换任何重复三次或多次的字符，重复的次数，然后是字符
- `replace_wrep`:: 用单词重复的特殊标记（`xxwrep`）替换任何重复三次或多次的单词，重复的次数，然后是单词
- `spec_add_spaces`:: 在 / 和 # 周围添加空格
- `rm_useless_spaces`:: 删除所有重复的空格字符
- `replace_all_caps`:: 将全部大写的单词小写，并在其前面添加所有大写的特殊标记 (`xxup`)
- `replace_maj`:: 小写大写单词并在其前面添加大写特殊标记 (`xxmaj`)
- `lowercase`:: 小写所有文本并在开头 (`xxbos`) 和/或结尾 (`xxeos`) 添加特殊标记

Let's take a look at a few of them in action:

让我们看一下其中的一些实例：

In [53]:
coll_repr(tkn('&copy;   Fast.ai www.fast.ai/INDEX'), 31)

"(#11) ['xxbos','©','xxmaj','fast.ai','xxrep','3','w','.fast.ai','/','xxup','index']"

Now let's take a look at how subword tokenization would work.

现在，让我们来看看子词标记化是如何工作的。

### Subword Tokenization

In addition to the *word tokenization* approach seen in the last section, another popular tokenization method is *subword tokenization*. Word tokenization relies on an assumption that spaces provide a useful separation of components of meaning in a sentence. However, this assumption is not always appropriate. For instance, consider this sentence: 我的名字是郝杰瑞 ("My name is Jeremy Howard" in Chinese). That's not going to work very well with a word tokenizer, because there are no spaces in it! Languages like Chinese and Japanese don't use spaces, and in fact they don't even have a well-defined concept of a "word." There are also languages, like Turkish and Hungarian, that can add many subwords together without spaces, creating very long words that include a lot of separate pieces of information.

To handle these cases, it's generally best to use subword tokenization. This proceeds in two steps:

1. Analyze a corpus of documents to find the most commonly occurring groups of letters. These become the vocab.
2. Tokenize the corpus using this vocab of *subword units*.

Let's look at an example. For our corpus, we'll use the first 2,000 movie reviews:

除了上一节中看到的`单词标记化`方法之外，另一种流行的标记化方法是`子词标记化`。单词标记化依赖于这样一个假设：空格提供了句子中含义成分的有用分离。然而，这种假设并不总是合适的。例如，考虑这句话：我的名字是郝杰瑞（中文`我的名字是杰里米·霍华德`）。这对于单词标记器来说效果不太好，因为其中没有空格！像中文和日语这样的语言不使用空格，事实上它们甚至没有明确定义的`单词`概念。还有一些语言，例如土耳其语和匈牙利语，可以将许多子词添加在一起而不使用空格，从而创建包含许多单独信息的非常长的单词。

为了处理这些情况，通常最好使用子词标记化。这分两步进行：

1. 分析文档语料库以查找最常出现的字母组。这些成为词汇。
2. 使用*子词单元*的词汇对语料库进行分词。

让我们看一个例子。对于我们的语料库，我们将使用前 2,000 条电影评论：

In [54]:
txts = L(o.open().read() for o in files[:2000])

We instantiate our tokenizer, passing in the size of the vocab we want to create, and then we need to "train" it. That is, we need to have it read our documents and find the common sequences of characters to create the vocab. This is done with `setup`. As we'll see shortly, `setup` is a special fastai method that is called automatically in our usual data processing pipelines. Since we're doing everything manually at the moment, however, we have to call it ourselves. Here's a function that does these steps for a given vocab size, and shows an example output:

我们实例化我们的分词器，传入我们想要创建的词汇的大小，然后我们需要`训练`它。也就是说，我们需要让它读取我们的文档并找到常见的字符序列来创建词汇。这是通过`设置`完成的。正如我们很快就会看到的，`setup`是一种特殊的 fastai 方法，在我们常用的数据处理管道中会自动调用。然而，由于目前我们正在手动完成所有操作，因此我们必须自己调用它。这是一个针对给定词汇大小执行这些步骤的函数，并显示了示例输出：

In [55]:
def subword(sz):
    sp = SubwordTokenizer(vocab_sz=sz)
    sp.setup(txts)
    return ' '.join(first(sp([txt]))[:40])

Let's try it out:

In [56]:
subword(1000)

'▁The ▁ra ci s m ▁in ▁this ▁dis g ra ce ful ▁production ▁a m id st ▁b re a th t ak ing ▁scene ry ▁follow s ▁the ▁pa t ter n ▁of ▁similar ▁production s ▁from ▁it s'

When using fastai's subword tokenizer, the special character `▁` represents a space character in the original text.

If we use a smaller vocab, then each token will represent fewer characters, and it will take more tokens to represent a sentence:

当使用fastai的子词分词器时，特殊字符``表示原始文本中的空格字符。

如果我们使用较小的词汇，那么每个标记将表示更少的字符，并且将需要更多的标记来表示一个句子：

In [57]:
subword(200)

'▁The ▁ ra c i s m ▁in ▁this ▁di s g ra ce f u l ▁p ro d u c t ion ▁a m i d st ▁b re a th t a k ing ▁ s c'

On the other hand, if we use a larger vocab, then most common English words will end up in the vocab themselves, and we will not need as many to represent a sentence:

另一方面，如果我们使用更大的词汇表，那么最常见的英语单词将最终出现在词汇表本身中，并且我们不需要那么多来表示一个句子：

In [58]:
subword(10000)

'▁The ▁racism ▁in ▁this ▁disgrace ful ▁production ▁a mid st ▁breathtaking ▁scenery ▁follows ▁the ▁pattern ▁of ▁similar ▁production s ▁from ▁its ▁source ▁and ▁time . ▁We ▁see ▁horde s ▁of ▁African s ▁slaughter ing ▁innocent ▁white s ▁and ▁are ▁bare'

Picking a subword vocab size represents a compromise: a larger vocab means fewer tokens per sentence, which means faster training, less memory, and less state for the model to remember; but on the downside, it means larger embedding matrices, which require more data to learn.

Overall, subword tokenization provides a way to easily scale between character tokenization (i.e., using a small subword vocab) and word tokenization (i.e., using a large subword vocab), and handles every human language without needing language-specific algorithms to be developed. It can even handle other "languages" such as genomic sequences or MIDI music notation! For this reason, in the last year its popularity has soared, and it seems likely to become the most common tokenization approach (it may well already be, by the time you read this!).

选择子词词汇大小代表一种折衷：较大的词汇意味着每个句子的标记较少，这意味着训练速度更快，内存更少，模型需要记住的状态也更少；但不利的一面是，这意味着更大的嵌入矩阵，需要更多的数据来学习。

总体而言，子词标记化提供了一种在字符标记化（即使用较小的子词词汇）和单词标记化（即使用较大的子词词汇）之间轻松扩展的方法，并且无需开发特定于语言的算法即可处理每种人类语言。它甚至可以处理其他`语言`，例如基因组序列或 MIDI 音乐符号！出于这个原因，去年它的受欢迎程度飙升，并且它似乎有可能成为最常见的标记化方法（当您阅读本文时，它很可能已经成为了！）。

Once our texts have been split into tokens, we need to convert them to numbers. We'll look at that next.

一旦我们的文本被分割成标记，我们需要将它们转换为数字。我们接下来会看看。

### Numericalization with fastai

*Numericalization* is the process of mapping tokens to integers. The steps are basically identical to those necessary to create a `Category` variable, such as the dependent variable of digits in MNIST:

1. Make a list of all possible levels of that categorical variable (the vocab).
1. Replace each level with its index in the vocab.

Let's take a look at this in action on the word-tokenized text we saw earlier:

*数值化*是将标记映射到整数的过程。这些步骤与创建`类别`变量所需的步骤基本相同，例如 MNIST 中数字的因变量：

1. 列出该分类变量（词汇）的所有可能级别。
1. 将每个级别替换为其在词汇表中的索引。

让我们看一下我们之前看到的单词标记化文本的实际效果：

In [59]:
toks = tkn(txt)
print(coll_repr(tkn(txt), 31))

(#161) ['xxbos','xxmaj','the','racism','in','this','disgraceful','production','amidst','breathtaking','scenery','follows','the','pattern','of','similar','productions','from','its','source','and','time','.','xxmaj','we','see','hordes','of','xxmaj','africans','slaughtering'...]


Just like with `SubwordTokenizer`, we need to call `setup` on `Numericalize`; this is how we create the vocab. That means we'll need our tokenized corpus first. Since tokenization takes a while, it's done in parallel by fastai; but for this manual walkthrough, we'll use a small subset:

就像`SubwordTokenizer`一样，我们需要在`Numericalize`上调用`setup`；这就是我们创建词汇的方式。这意味着我们首先需要我们的标记化语料库。由于标记化需要一段时间，因此由 fastai 并行完成；但对于本手动演练，我们将使用一小部分：

In [60]:
toks200 = txts[:200].map(tkn)
toks200[0]

(#161) ['xxbos','xxmaj','the','racism','in','this','disgraceful','production','amidst','breathtaking'...]

We can pass this to `setup` to create our vocab:

我们可以将其传递给 `setup` 来创建我们的词汇表：

In [61]:
num = Numericalize()
num.setup(toks200)
coll_repr(num.vocab,20)

"(#1992) ['xxunk','xxpad','xxbos','xxeos','xxfld','xxrep','xxwrep','xxup','xxmaj','the',',','.','and','a','of','to','is','it','in','i'...]"

Our special rules tokens appear first, and then every word appears once, in frequency order. The defaults to `Numericalize` are `min_freq=3,max_vocab=60000`. `max_vocab=60000` results in fastai replacing all words other than the most common 60,000 with a special *unknown word* token, `xxunk`. This is useful to avoid having an overly large embedding matrix, since that can slow down training and use up too much memory, and can also mean that there isn't enough data to train useful representations for rare words. However, this last issue is better handled by setting `min_freq`; the default `min_freq=3` means that any word appearing less than three times is replaced with `xxunk`.

fastai can also numericalize your dataset using a vocab that you provide, by passing a list of words as the `vocab` parameter.

Once we've created our `Numericalize` object, we can use it as if it were a function:

我们的特殊规则标记会首先出现，然后每个单词都会按频率顺序出现一次。`Numericalize` 的默认值是 `min_freq=3,max_vocab=60000`。`max_vocab=60000`会导致 fastai 用一个特殊的 *unknown word* 标记 "xxxunk "替换最常见的 60,000 个单词以外的所有单词。这有助于避免嵌入矩阵过大，因为这会减慢训练速度并占用过多内存，同时也意味着没有足够的数据来训练有用的稀有词表示。不过，通过设置 `min_freq` 可以更好地处理最后一个问题；默认的 `min_freq=3` 意味着任何出现次数少于三次的词都会被替换为 `xxxunk`。

fastai 还可以使用你提供的词汇表对数据集进行数值化，方法是将词汇列表作为 `vocab` 参数传递。

创建了 `Numericalize` 对象后，我们就可以像使用函数一样使用它了：

In [62]:
nums = num(toks)[:20]; nums

TensorText([   2,    8,    9,    0,   18,   20,    0,  398,    0,    0,  798, 1152,    9,    0,   14,  611,    0,   55,  125, 1453])

This time, our tokens have been converted to a tensor of integers that our model can receive. We can check that they map back to the original text:

这次，我们的令牌已转换为我们的模型可以接收的整数张量。我们可以检查它们是否映射回原始文本：

In [63]:
' '.join(num.vocab[o] for o in nums)

'xxbos xxmaj the xxunk in this xxunk production xxunk xxunk scenery follows the xxunk of similar xxunk from its source'

Now that we have numbers, we need to put them in batches for our model.

现在我们有了数字，我们需要将它们分批放入我们的模型中。

### Putting Our Texts into Batches for a Language Model

When dealing with images, we needed to resize them all to the same height and width before grouping them together in a mini-batch so they could stack together efficiently in a single tensor. Here it's going to be a little different, because one cannot simply resize text to a desired length. Also, we want our language model to read text in order, so that it can efficiently predict what the next word is. This means that each new batch should begin precisely where the previous one left off.

Suppose we have the following text:

> : In this chapter, we will go back over the example of classifying movie reviews we studied in chapter 1 and dig deeper under the surface. First we will look at the processing steps necessary to convert text into numbers and how to customize it. By doing this, we'll have another example of the PreProcessor used in the data block API.\nThen we will study how we build a language model and train it for a while.

The tokenization process will add special tokens and deal with punctuation to return this text:

> : xxbos xxmaj in this chapter , we will go back over the example of classifying movie reviews we studied in chapter 1 and dig deeper under the surface . xxmaj first we will look at the processing steps necessary to convert text into numbers and how to customize it . xxmaj by doing this , we 'll have another example of the preprocessor used in the data block xxup api . \n xxmaj then we will study how we build a language model and train it for a while .

We now have 90 tokens, separated by spaces. Let's say we want a batch size of 6. We need to break this text into 6 contiguous parts of length 15:

处理图像时，我们需要将它们全部调整为相同的高度和宽度，然后将它们分组到一个小批量中，以便它们可以在单个张量中有效地堆叠在一起。这里会有点不同，因为不能简单地将文本大小调整到所需的长度。此外，我们希望我们的语言模型能够按顺序读取文本，以便它可以有效地预测下一个单词是什么。这意味着每个新批次都应准确地从上一个批次停止的位置开始。

假设我们有以下文本：

>：在本章中，我们将回顾我们在第一章中学习的电影评论分类示例，并深入挖掘表面之下。首先，我们将了解将文本转换为数字所需的处理步骤以及如何对其进行自定义。通过这样做，我们将获得数据块 API 中使用的预处理器的另一个示例。\n然后我们将研究如何构建语言模型并对其进行一段时间的训练。

标记化过程将添加特殊标记并处理标点符号以返回以下文本：

> : xxbos xxmaj 在本章中，我们将回顾我们在第一章中学习的电影评论分类示例，并深入挖掘表面之下。 xxmaj 首先，我们将了解将文本转换为数字所需的处理步骤以及如何对其进行自定义。 xxmaj 通过这样做，我们将获得数据块 xxup api 中使用的预处理器的另一个示例。 \n xxmaj 然后我们将研究如何构建语言模型并对其进行一段时间的训练。

我们现在有 90 个令牌，用空格分隔。假设我们想要的批量大小为 6。我们需要将此文本分成 6 个长度为 15 的连续部分：

In [64]:
#hide_input
stream = "In this chapter, we will go back over the example of classifying movie reviews we studied in chapter 1 and dig deeper under the surface. First we will look at the processing steps necessary to convert text into numbers and how to customize it. By doing this, we'll have another example of the PreProcessor used in the data block API.\nThen we will study how we build a language model and train it for a while."
tokens = tkn(stream)
bs,seq_len = 6,15
d_tokens = np.array([tokens[i*seq_len:(i+1)*seq_len] for i in range(bs)])
df = pd.DataFrame(d_tokens)
display(HTML(df.to_html(index=False,header=None)))

xxbos,xxmaj,in,this,chapter,",",we,will,go,back,over,the,example,of,classifying
movie,reviews,we,studied,in,chapter,1,and,dig,deeper,under,the,surface,.,xxmaj
first,we,will,look,at,the,processing,steps,necessary,to,convert,text,into,numbers,and
how,to,customize,it,.,xxmaj,by,doing,this,",",we,'ll,have,another,example
of,the,preprocessor,used,in,the,data,block,xxup,api,.,\n,xxmaj,then,we
will,study,how,we,build,a,language,model,and,train,it,for,a,while,.


In a perfect world, we could then give this one batch to our model. But that approach doesn't scale, because outside of this toy example it's unlikely that a single batch containing all the texts would fit in our GPU memory (here we have 90 tokens, but all the IMDb reviews together give several million).

So, we need to divide this array more finely into subarrays of a fixed sequence length. It is important to maintain order within and across these subarrays, because we will use a model that maintains a state so that it remembers what it read previously when predicting what comes next. 

Going back to our previous example with 6 batches of length 15, if we chose a sequence length of 5, that would mean we first feed the following array:

在完美的世界中，我们可以将这一批交给我们的模型。但这种方法无法扩展，因为在这个玩具示例之外，包含所有文本的单个批次不太可能适合我们的 GPU 内存（这里我们有 90 个令牌，但所有 IMDb 评论加起来有几百万个）。

因此，我们需要将这个数组更精细地划分为固定序列长度的子数组。维护这些子数组内部和子数组之间的顺序非常重要，因为我们将使用一个维护状态的模型，以便在预测接下来发生的情况时记住之前读取的内容。 

回到我们之前的例子，有 6 个批次，长度为 15，如果我们选择序列长度为 5，这意味着我们首先输入以下数组：

In [65]:
#hide_input
bs,seq_len = 6,5
d_tokens = np.array([tokens[i*15:i*15+seq_len] for i in range(bs)])
df = pd.DataFrame(d_tokens)
display(HTML(df.to_html(index=False,header=None)))

xxbos,xxmaj,in,this,chapter
movie,reviews,we,studied,in
first,we,will,look,at
how,to,customize,it,.
of,the,preprocessor,used,in
will,study,how,we,build


Then this one:

In [66]:
#hide_input
bs,seq_len = 6,5
d_tokens = np.array([tokens[i*15+seq_len:i*15+2*seq_len] for i in range(bs)])
df = pd.DataFrame(d_tokens)
display(HTML(df.to_html(index=False,header=None)))

",",we,will,go,back
chapter,1,and,dig,deeper
the,processing,steps,necessary,to
xxmaj,by,doing,this,","
the,data,block,xxup,api
a,language,model,and,train


And finally:

In [67]:
#hide_input
bs,seq_len = 6,5
d_tokens = np.array([tokens[i*15+10:i*15+15] for i in range(bs)])
df = pd.DataFrame(d_tokens)
display(HTML(df.to_html(index=False,header=None)))

over,the,example,of,classifying
under,the,surface,.,xxmaj
convert,text,into,numbers,and
we,'ll,have,another,example
.,\n,xxmaj,then,we
it,for,a,while,.


Going back to our movie reviews dataset, the first step is to transform the individual texts into a stream by concatenating them together. As with images, it's best to randomize the order of the inputs, so at the beginning of each epoch we will shuffle the entries to make a new stream (we shuffle the order of the documents, not the order of the words inside them, or the texts would not make sense anymore!).

We then cut this stream into a certain number of batches (which is our *batch size*). For instance, if the stream has 50,000 tokens and we set a batch size of 10, this will give us 10 mini-streams of 5,000 tokens. What is important is that we preserve the order of the tokens (so from 1 to 5,000 for the first mini-stream, then from 5,001 to 10,000...), because we want the model to read continuous rows of text (as in the preceding example). An `xxbos` token is added at the start of each during preprocessing, so that the model knows when it reads the stream when a new entry is beginning.

So to recap, at every epoch we shuffle our collection of documents and concatenate them into a stream of tokens. We then cut that stream into a batch of fixed-size consecutive mini-streams. Our model will then read the mini-streams in order, and thanks to an inner state, it will produce the same activation whatever sequence length we picked.

This is all done behind the scenes by the fastai library when we create an `LMDataLoader`. We do this by first applying our `Numericalize` object to the tokenized texts:

回到我们的电影评论数据集，第一步是将各个文本连接在一起，将它们转换为流。与图像一样，最好随机化输入的顺序，因此在每个纪元开始时，我们将打乱条目以创建一个新流（我们打乱文档的顺序，而不是其中单词的顺序，或者文本将不再有意义！）。

然后，我们将此流切割成一定数量的批次（这是我们的*批次大小*）。例如，如果流有 50,000 个令牌，并且我们将批量大小设置为 10，则这将为我们提供 10 个包含 5,000 个令牌的迷你流。重要的是我们保留标记的顺序（因此第一个迷你流从 1 到 5,000，然后从 5,001 到 10,000...），因为我们希望模型读取连续的文本行（如前面的例子）。在预处理期间，在每个开始处添加一个`xxbos`标记，以便模型知道在新条目开始时何时读取流。

回顾一下，在每个时期，我们都会对文档集合进行洗牌，并将它们连接成令牌流。然后，我们将该流切割成一批固定大小的连续迷你流。然后，我们的模型将按顺序读取迷你流，并且由于内部状态，无论我们选择的序列长度如何，它都会产生相同的激活。

当我们创建`LMDataLoader`时，这一切都是由 fastai 库在幕后完成的。我们首先将`Numericalize`对象应用于标记化文本：

In [68]:
nums200 = toks200.map(num)

and then passing that to `LMDataLoader`:

In [69]:
dl = LMDataLoader(nums200)

Let's confirm that this gives the expected results, by grabbing the first batch:

让我们通过获取第一批来确认这是否给出了预期的结果：

In [70]:
x,y = first(dl)
x.shape,y.shape

(torch.Size([64, 72]), torch.Size([64, 72]))

and then looking at the first row of the independent variable, which should be the start of the first text:

然后查看自变量的第一行，它应该是第一个文本的开头：

In [71]:
' '.join(num.vocab[o] for o in x[0][:20])

'xxbos xxmaj the xxunk in this xxunk production xxunk xxunk scenery follows the xxunk of similar xxunk from its source'

The dependent variable is the same thing offset by one token:

因变量是相同的东西，偏移一个标记：

In [72]:
' '.join(num.vocab[o] for o in y[0][:20])

'xxmaj the xxunk in this xxunk production xxunk xxunk scenery follows the xxunk of similar xxunk from its source and'

This concludes all the preprocessing steps we need to apply to our data. We are now ready to train our text classifier.

这就结束了我们需要应用于数据的所有预处理步骤。我们现在准备训练我们的文本分类器。

## Training a Text Classifier

As we saw at the beginning of this chapter, there are two steps to training a state-of-the-art text classifier using transfer learning: first we need to fine-tune our language model pretrained on Wikipedia to the corpus of IMDb reviews, and then we can use that model to train a classifier.

As usual, let's start with assembling our data.

正如我们在本章开头所看到的，使用迁移学习训练最先进的文本分类器有两个步骤：首先，我们需要将在维基百科上预训练的语言模型微调到 IMDb 评论语料库，然后我们可以使用该模型来训练分类器。

像往常一样，让我们​​从组装数据开始。

### Language Model Using DataBlock

fastai handles tokenization and numericalization automatically when `TextBlock` is passed to `DataBlock`. All of the arguments that can be passed to `Tokenize` and `Numericalize` can also be passed to `TextBlock`. In the next chapter we'll discuss the easiest ways to run each of these steps separately, to ease debugging—but you can always just debug by running them manually on a subset of your data as shown in the previous sections. And don't forget about `DataBlock`'s handy `summary` method, which is very useful for debugging data issues.

Here's how we use `TextBlock` to create a language model, using fastai's defaults:

当`TextBlock`传递给`DataBlock`时，fastai 会自动处理标记化和数字化。所有可以传递给`Tokenize`和`Numericalize`的参数也可以传递给`TextBlock`。在下一章中，我们将讨论单独运行每个步骤的最简单方法，以简化调试，但您始终可以通过在数据子集上手动运行它们来进行调试，如前面部分所示。并且不要忘记`DataBlock`方便的`summary`方法，它对于调试数据问题非常有用。

以下是我们如何使用 `TextBlock` 创建语言模型，使用 fastai 的默认值：

In [73]:
get_imdb = partial(get_text_files, folders=['train', 'test', 'unsup'])

dls_lm = DataBlock(
    blocks=TextBlock.from_folder(path, is_lm=True),
    get_items=get_imdb, splitter=RandomSplitter(0.1)
).dataloaders(path, path=path, bs=128, seq_len=80)

One thing that's different to previous types we've used in `DataBlock` is that we're not just using the class directly (i.e., `TextBlock(...)`, but instead are calling a *class method*. A class method is a Python method that, as the name suggests, belongs to a *class* rather than an *object*. (Be sure to search online for more information about class methods if you're not familiar with them, since they're commonly used in many Python libraries and applications; we've used them a few times previously in the book, but haven't called attention to them.) The reason that `TextBlock` is special is that setting up the numericalizer's vocab can take a long time (we have to read and tokenize every document to get the vocab). To be as efficient as possible it performs a few optimizations: 

- It saves the tokenized documents in a temporary folder, so it doesn't have to tokenize them more than once
- It runs multiple tokenization processes in parallel, to take advantage of your computer's CPUs

We need to tell `TextBlock` how to access the texts, so that it can do this initial preprocessing—that's what `from_folder` does.

`show_batch` then works in the usual way:

与我们之前在`DataBlock`中使用的类型不同的一件事是，我们不只是直接使用该类（即`TextBlock(...)`，而是调用*类方法*。 method 是一个 Python 方法，顾名思义，属于 *class* 而不是 *object*（如果您不熟悉它们，请务必在线搜索有关类方法的更多信息，因为它们是）。在许多 Python 库和应用程序中常用；我们在本书之前已经使用过它们几次，但没有引起人们的注意。）`TextBlock`特殊的原因是设置数值化器的词汇可能需要很长时间（我们必须读取并标记每个文档才能获取词汇）。为了尽可能高效，它执行了一些优化： 

- 它将标记化文档保存在临时文件夹中，因此不必多次标记它们
- 它并行运行多个标记化进程，以利用计算机的 CPU

我们需要告诉`TextBlock`如何访问文本，以便它可以进行初始预处理——这就是`from_folder`所做的。

然后 `show_batch` 以通常的方式工作：

In [74]:
dls_lm.show_batch(max_n=2)

,text,text_
0,"xxbos xxmaj not wishing to give * anything * away here , i would just say this technically excellent , flawlessly acted and uplifting little flic will reward the viewer with an excellent hour and a half 's entertainment : xxmaj it will amuse , surprise , possibly embarrass occasionally and almost certainly tug at the heartstrings from time to time , as it approaches the inevitable , but not obvious , ending without becoming clichéd or predictable in any","xxmaj not wishing to give * anything * away here , i would just say this technically excellent , flawlessly acted and uplifting little flic will reward the viewer with an excellent hour and a half 's entertainment : xxmaj it will amuse , surprise , possibly embarrass occasionally and almost certainly tug at the heartstrings from time to time , as it approaches the inevitable , but not obvious , ending without becoming clichéd or predictable in any way"
1,". xxmaj they 're a couple with difficulties and hope that the holiday will spice up their life a bit . \n\n xxmaj but they have the unfortunate luck of meeting xxmaj oscar and xxmaj mimi , an awkward couple . xxmaj the temptress xxmaj mimi flirts with xxmaj nigel , much to xxmaj fiona 's disgust . xxmaj and the wheelchair - bound xxmaj oscar sees in xxmaj nigel the perfect listener to his life story . \n\n xxmaj","xxmaj they 're a couple with difficulties and hope that the holiday will spice up their life a bit . \n\n xxmaj but they have the unfortunate luck of meeting xxmaj oscar and xxmaj mimi , an awkward couple . xxmaj the temptress xxmaj mimi flirts with xxmaj nigel , much to xxmaj fiona 's disgust . xxmaj and the wheelchair - bound xxmaj oscar sees in xxmaj nigel the perfect listener to his life story . \n\n xxmaj much"


Now that our data is ready, we can fine-tune the pretrained language model.

### Fine-Tuning the Language Model

To convert the integer word indices into activations that we can use for our neural network, we will use embeddings, just like we did for collaborative filtering and tabular modeling. Then we'll feed those embeddings into a *recurrent neural network* (RNN), using an architecture called *AWD-LSTM* (we will show you how to write such a model from scratch in <<chapter_nlp_dive>>). As we discussed earlier, the embeddings in the pretrained model are merged with random embeddings added for words that weren't in the pretraining vocabulary. This is handled automatically inside `language_model_learner`:

为了将整数单词索引转换为可用于神经网络的激活，我们将使用嵌入，就像我们在协作过滤和表格建模中所做的那样。然后，我们将使用名为 *AWD-LSTM* 的架构将这些嵌入输入*循环神经网络* (RNN)（我们将在 <<chapter_nlp_dive>> 中向您展示如何从头开始编写这样的模型）。正如我们之前讨论的，预训练模型中的嵌入与为不在预训练词汇表中的单词添加的随机嵌入合并。这是在`language_model_learner`内部自动处理的：

In [75]:
learn = language_model_learner(
    dls_lm, AWD_LSTM, drop_mult=0.3, 
    metrics=[accuracy, Perplexity()]).to_fp16()

The loss function used by default is cross-entropy loss, since we essentially have a classification problem (the different categories being the words in our vocab). The *perplexity* metric used here is often used in NLP for language models: it is the exponential of the loss (i.e., `torch.exp(cross_entropy)`). We  also include the accuracy metric, to see how many times our model is right when trying to predict the next word, since cross-entropy (as we've seen) is both hard to interpret, and tells us more about the model's confidence than its accuracy.

Let's go back to the process diagram from the beginning of this chapter. The first arrow has been completed for us and made available as a pretrained model in fastai, and we've just built the `DataLoaders` and `Learner` for the second stage. Now we're ready to fine-tune our language model!

默认使用的损失函数是交叉熵损失，因为我们本质上有一个分类问题（不同的类别是我们词汇中的单词）。这里使用的 *perplexity* 度量通常在 NLP 语言模型中使用：它是损失的指数（即`torch.exp(cross_entropy)`）。我们还包括准确性指标，以查看我们的模型在尝试预测下一个单词时正确的次数，因为交叉熵（正如我们所见）既难以解释，又比交叉熵更能告诉我们模型的置信度。它的准确性。

让我们回到本章开头的流程图。第一个箭头已经为我们完成，并可作为 fastai 中的预训练模型使用，我们刚刚为第二阶段构建了`DataLoaders`和`Learner`。现在我们准备好调整我们的语言模型了！

<img alt="Diagram of the ULMFiT process" width="450" src="images/att_00027.png">

It takes quite a while to train each epoch, so we'll be saving the intermediate model results during the training process. Since `fine_tune` doesn't do that for us, we'll use `fit_one_cycle`. Just like `vision_learner`, `language_model_learner` automatically calls `freeze` when using a pretrained model (which is the default), so this will only train the embeddings (the only part of the model that contains randomly initialized weights—i.e., embeddings for words that are in our IMDb vocab, but aren't in the pretrained model vocab):

训练每个 epoch 需要相当长的时间，因此我们将在训练过程中保存中间模型结果。由于`fine_tune`不能为我们做到这一点，因此我们将使用`fit_one_cycle`。就像`vision_learner`一样，`language_model_learner`在使用预训练模型（默认情况下）时会自动调用`freeze`，因此这只会训练嵌入（模型中包含随机初始化权重的唯一部分，即嵌入存在于我们的 IMDb 词汇中但不在预训练模型词汇中的单词）：

In [76]:
learn.fit_one_cycle(1, 2e-2)

epoch,train_loss,valid_loss,accuracy,perplexity,time
0,6.805713,6.449951,0.127585,632.671082,19:16


This model takes a while to train, so it's a good opportunity to talk about saving intermediary results. 

这个模型需要一段时间来训练，所以这是一个谈论保存中间结果的好机会。

### Saving and Loading Models

You can easily save the state of your model like so:

您可以轻松保存模型的状态，如下所示：

In [80]:
learn.save('1epoch')

Path('/home/zjb/.fastai/data/imdb/models/1epoch.pth')

This will create a file in `learn.path/models/` named *1epoch.pth*. If you want to load your model in another machine after creating your `Learner` the same way, or resume training later, you can load the content of this file with:

这将在`learn.path/models/`中创建一个名为 *1epoch.pth* 的文件。如果您想在以相同方式创建`Learner`后将模型加载到另一台计算机中，或者稍后恢复训练，您可以使用以下命令加载此文件的内容：

In [81]:
learn = learn.load('1epoch')

Once the initial training has completed, we can continue fine-tuning the model after unfreezing:

初始训练完成后，我们可以在解冻后继续对模型进行微调：

In [82]:
learn.unfreeze()
learn.fit_one_cycle(10, 2e-3)

epoch,train_loss,valid_loss,accuracy,perplexity,time
0,4.630449,4.545200,0.260312,94.179253,20:36
1,4.440998,4.358315,0.280459,78.125343,20:30
2,4.319020,4.266542,0.291764,71.274773,20:33
3,4.258798,4.209347,0.298044,67.312553,20:41
4,4.208580,4.167527,0.303147,64.555618,20:35
5,4.143332,4.130752,0.307702,62.224705,20:53
6,4.107087,4.101960,0.310820,60.458683,20:43
7,4.058275,4.082058,0.313638,59.267315,20:43
8,4.020222,4.072774,0.314792,58.719620,20:36
9,4.009973,4.071673,0.314967,58.655033,20:37


Once this is done, we save all of our model except the final layer that converts activations to probabilities of picking each token in our vocabulary. The model not including the final layer is called the *encoder*. We can save it with `save_encoder`:

完成此操作后，我们保存所有模型，除了最后一层，该层将激活转换为选择词汇表中每个标记的概率。不包括最后一层的模型称为*编码器*。我们可以使用`save_encoder`保存它：

In [83]:
learn.save_encoder('finetuned')

> jargon: Encoder: The model not including the task-specific final layer(s). This term means much the same thing as _body_ when applied to vision CNNs, but "encoder" tends to be more used for NLP and generative models.

> 行话：编码器：不包括特定于任务的最终层的模型。当应用于视觉 CNN 时，该术语与 _body_ 的含义大致相同，但`编码器`往往更多地用于 NLP 和生成模型。

This completes the second stage of the text classification process: fine-tuning the language model. We can now use it to fine-tune a classifier using the IMDb sentiment labels.

这就完成了文本分类过程的第二阶段：微调语言模型。我们现在可以使用它来使用 IMDb 情感标签来微调分类器。

### Text Generation

Before we move on to fine-tuning the classifier, let's quickly try something different: using our model to generate random reviews. Since it's trained to guess what the next word of the sentence is, we can use the model to write new reviews:

在我们继续微调分类器之前，让我们快速尝试一些不同的事情：使用我们的模型生成随机评论。由于它被训练来猜测句子的下一个单词是什么，我们可以使用该模型来撰写新评论：

In [84]:
TEXT = "I liked this movie because"
N_WORDS = 40
N_SENTENCES = 2
preds = [learn.predict(TEXT, N_WORDS, temperature=0.75) 
         for _ in range(N_SENTENCES)]

In [87]:
print("\n".join(preds))

i liked this movie because i loved it . It was my favorite movie and it was the last one i loved . Every time i watch it i still feel connected . i just ca nt believe how bad this movie was
i liked this movie because it was completely different from the James Bond movies . The title character as well as the main character is , really , the Italian . i do tomme a lot , but i think it


As you can see, we add some randomness (we pick a random word based on the probabilities returned by the model) so we don't get exactly the same review twice. Our model doesn't have any programmed knowledge of the structure of a sentence or grammar rules, yet it has clearly learned a lot about English sentences: we can see it capitalizes properly (*I* is just transformed to *i* because our rules require two characters or more to consider a word as capitalized, so it's normal to see it lowercased) and is using consistent tense. The general review makes sense at first glance, and it's only if you read carefully that you can notice something is a bit off. Not bad for a model trained in a couple of hours! 

But our end goal wasn't to train a model to generate reviews, but to classify them... so let's use this model to do just that.

正如您所看到的，我们添加了一些随机性（我们根据模型返回的概率选择一个随机单词），因此我们不会两次获得完全相同的评论。我们的模型没有任何关于句子结构或语法规则的编程知识，但它显​​然已经学到了很多关于英语句子的知识：我们可以看到它正确地大写（*I* 只是转换为 *i* 因为我们的规则需要两个或更多字符才能将单词视为大写，因此看到它是小写是正常的）并且使用一致的时态。总体评论乍一看很有道理，只有仔细阅读才能发现有些不对劲。对于一个在几个小时内训练出来的模型来说已经不错了！ 

但我们的最终目标不是训练模型来生成评论，而是对它们进行分类……所以让我们使用这个模型来做到这一点。

### Creating the Classifier DataLoaders

We're now moving from language model fine-tuning to classifier fine-tuning. To recap, a language model predicts the next word of a document, so it doesn't need any external labels. A classifier, however, predicts some external label—in the case of IMDb, it's the sentiment of a document.

This means that the structure of our `DataBlock` for NLP classification will look very familiar. It's actually nearly the same as we've seen for the many image classification datasets we've worked with:

我们现在正在从语言模型微调转向分类器微调。回顾一下，语言模型可以预测文档的下一个单词，因此它不需要任何外部标签。然而，分类器可以预测一些外部标签——在 IMDb 中，它是文档的情感。

这意味着我们用于 NLP 分类的`DataBlock`结构看起来会非常熟悉。它实际上与我们在使用的许多图像分类数据集上看到的几乎相同：

In [88]:
dls_clas = DataBlock(
    blocks=(TextBlock.from_folder(path, vocab=dls_lm.vocab),CategoryBlock),
    get_y = parent_label,
    get_items=partial(get_text_files, folders=['train', 'test']),
    splitter=GrandparentSplitter(valid_name='test')
).dataloaders(path, path=path, bs=128, seq_len=72)

Just like with image classification, `show_batch` shows the dependent variable (sentiment, in this case) with each independent variable (movie review text):

就像图像分类一样，`show_batch`显示因变量（在本例中为情绪）与每个自变量（电影评论文本）：

In [89]:
dls_clas.show_batch(max_n=3)

,text,category
0,"xxbos xxmaj match 1 : xxmaj tag xxmaj team xxmaj table xxmaj match xxmaj bubba xxmaj ray and xxmaj spike xxmaj dudley vs xxmaj eddie xxmaj guerrero and xxmaj chris xxmaj benoit xxmaj bubba xxmaj ray and xxmaj spike xxmaj dudley started things off with a xxmaj tag xxmaj team xxmaj table xxmaj match against xxmaj eddie xxmaj guerrero and xxmaj chris xxmaj benoit . xxmaj according to the rules of the match , both opponents have to go through tables in order to get the win . xxmaj benoit and xxmaj guerrero heated up early on by taking turns hammering first xxmaj spike and then xxmaj bubba xxmaj ray . a xxmaj german xxunk by xxmaj benoit to xxmaj bubba took the wind out of the xxmaj dudley brother . xxmaj spike tried to help his brother , but the referee restrained him while xxmaj benoit and xxmaj guerrero",pos
1,"xxbos * ! ! - xxup spoilers - ! ! * \n\n xxmaj before i begin this , let me say that i have had both the advantages of seeing this movie on the big screen and of having seen the "" authorized xxmaj version "" of this movie , remade by xxmaj stephen xxmaj king , himself , in 1997 . \n\n xxmaj both advantages made me appreciate this version of "" the xxmaj shining , "" all the more . \n\n xxmaj also , let me say that xxmaj i 've read xxmaj mr . xxmaj king 's book , "" the xxmaj shining "" on many occasions over the years , and while i love the book and am a huge fan of his work , xxmaj stanley xxmaj kubrick 's retelling of this story is far more compelling … and xxup scary . \n\n xxmaj kubrick",pos
2,"xxbos xxmaj berlin - born in 1942 xxmaj margarethe von xxmaj trotta was an actress and now she is a very important director and writer . xxmaj she has been described , perhaps even unfairly caricatured , as a director whose commitment to bringing a woman 's sensibility to the screen outweighs her artistic strengths . "" rosenstrasse , "" which has garnered mixed and even strange reviews ( the xxmaj new xxmaj york xxmaj times article was one of the most negatively aggressive reviews xxmaj i 've ever read in that paper ) is not a perfect film . xxmaj it is a fine movie and a testament to a rare xxunk of successful opposition to the genocidal xxmaj nazi regime by , of all peoples , generically powerless xxmaj germans demonstrating in a xxmaj berlin street . \n\n xxmaj co - writer von xxmaj trotta uses the actual",pos


Looking at the `DataBlock` definition, every piece is familiar from previous data blocks we've built, with two important exceptions:

- `TextBlock.from_folder` no longer has the `is_lm=True` parameter.
- We pass the `vocab` we created for the language model fine-tuning.

The reason that we pass the `vocab` of the language model is to make sure we use the same correspondence of token to index. Otherwise the embeddings we learned in our fine-tuned language model won't make any sense to this model, and the fine-tuning step won't be of any use.

By passing `is_lm=False` (or not passing `is_lm` at all, since it defaults to `False`) we tell `TextBlock` that we have regular labeled data, rather than using the next tokens as labels. There is one challenge we have to deal with, however, which is to do with collating multiple documents into a mini-batch. Let's see with an example, by trying to create a mini-batch containing the first 10 documents. First we'll numericalize them:

查看`DataBlock`定义，每个部分都与我们之前构建的数据块相似，但有两个重要的例外：

- `TextBlock.from_folder` 不再具有 `is_lm=True` 参数。
- 我们传递我们为语言模型微调创建的`vocab`。

我们传递语言模型的`vocab`的原因是为了确保我们使用相同的标记对应关系来索引。否则，我们在微调语言模型中学到的嵌入对该模型没有任何意义，并且微调步骤也没有任何用处。

通过传递`is_lm=False`（或者根本不传递`is_lm`，因为它默认为`False`），我们告诉`TextBlock`我们有常规标记数据，而不是使用下一个标记作为标签。然而，我们必须应对一个挑战，即将多个文档整理成一个小批次。让我们看一个示例，尝试创建一个包含前 10 个文档的小批量。首先我们将它们数值化：

In [90]:
nums_samp = toks200[:10].map(num)

Let's now look at how many tokens each of these 10 movie reviews have:

现在让我们看看这 10 条影评分别有多少个token：

In [91]:
nums_samp.map(len)

(#10) [161,441,152,154,285,161,204,508,398,107]

Remember, PyTorch `DataLoader`s need to collate all the items in a batch into a single tensor, and a single tensor has a fixed shape (i.e., it has some particular length on every axis, and all items must be consistent). This should sound familiar: we had the same issue with images. In that case, we used cropping, padding, and/or squishing to make all the inputs the same size. Cropping might not be a good idea for documents, because it seems likely we'd remove some key information (having said that, the same issue is true for images, and we use cropping there; data augmentation hasn't been well explored for NLP yet, so perhaps there are actually opportunities to use cropping in NLP too!). You can't really "squish" a document. So that leaves padding!

We will expand the shortest texts to make them all the same size. To do this, we use a special padding token that will be ignored by our model. Additionally, to avoid memory issues and improve performance, we will batch together texts that are roughly the same lengths (with some shuffling for the training set). We do this by (approximately, for the training set) sorting the documents by length prior to each epoch. The result of this is that the documents collated into a single batch will tend to be of similar lengths. We won't pad every batch to the same size, but will instead use the size of the largest document in each batch as the target size. (It is possible to do something similar with images, which is especially useful for irregularly sized rectangular images, but at the time of writing no library provides good support for this yet, and there aren't any papers covering it. It's something we're planning to add to fastai soon, however, so keep an eye on the book's website; we'll add information about this as soon as we have it working well.)

The sorting and padding are automatically done by the data block API for us when using a `TextBlock`, with `is_lm=False`. (We don't have this same issue for language model data, since we concatenate all the documents together first, and then split them into equally sized sections.)

We can now create a model to classify our texts:

请记住，PyTorch `DataLoader` 需要将批次中的所有项目整理成单个张量，并且单个张量具有固定形状（即，它在每个轴上具有特定的长度，并且所有项目必须一致）。这听起来应该很熟悉：我们在图像方面也遇到了同样的问题。在这种情况下，我们使用裁剪、填充和/或挤压来使所有输入具有相同的大小。对于文档来说，裁剪可能不是一个好主意，因为我们可能会删除一些关键信息（话虽如此，对于图像也存在同样的问题，我们在那里使用裁剪；数据增强尚未在 NLP 中得到很好的探索）然而，也许实际上也有机会在 NLP 中使用裁剪！）。您无法真正“压缩”文档。这样就剩下填充了！

我们将扩展最短的文本，使它们都具有相同的大小。为此，我们使用一个特殊的填充标记，该标记将被我们的模型忽略。此外，为了避免内存问题并提高性能，我们将把长度大致相同的文本分批在一起（对训练集进行一些改组）。我们通过（大约对于训练集）在每个纪元之前按长度对文档进行排序来做到这一点。这样做的结果是整理成单个批次的文档往往具有相似的长度。我们不会将每个批次填充到相同的大小，而是使用每个批次中最大文档的大小作为目标大小。 （可以对图像做类似的事情，这对于不规则大小的矩形图像特别有用，但在撰写本文时，还没有库对此提供良好的支持，并且没有任何论文涵盖它。这是我们的事情不过，我们计划很快将其添加到 fastai，因此请密切关注该书的网站，一旦运行良好，我们将立即添加相关信息。）

当使用 `TextBlock` 且 `is_lm=False` 时，数据块 API 会自动为我们完成排序和填充。 （对于语言模型数据，我们没有同样的问题，因为我们首先将所有文档连接在一起，然后将它们分成相同大小的部分。）

我们现在可以创建一个模型来对文本进行分类：

In [92]:
learn = text_classifier_learner(dls_clas, AWD_LSTM, drop_mult=0.5, 
                                metrics=accuracy).to_fp16()

The final step prior to training the classifier is to load the encoder from our fine-tuned language model. We use `load_encoder` instead of `load` because we only have pretrained weights available for the encoder; `load` by default raises an exception if an incomplete model is loaded:

训练分类器之前的最后一步是从我们微调的语言模型加载编码器。我们使用`load_encoder`而不是`load`，因为我们只有预训练的权重可用于编码器；如果加载不完整的模型，`load`默认会引发异常：

In [93]:
learn = learn.load_encoder('finetuned')

### Fine-Tuning the Classifier

The last step is to train with discriminative learning rates and *gradual unfreezing*. In computer vision we often unfreeze the model all at once, but for NLP classifiers, we find that unfreezing a few layers at a time makes a real difference:

最后一步是使用有区别的学习率和“逐渐解冻”进行训练。在计算机视觉中，我们经常一次性解冻模型，但对于 NLP 分类器，我们发现一次解冻几个层会产生真正的区别：

In [94]:
learn.fit_one_cycle(1, 2e-2)

epoch,train_loss,valid_loss,accuracy,time
0,0.425562,0.339746,0.859680,01:13


In just one epoch we get the same result as our training in <<chapter_intro>>: not too bad! We can pass `-2` to `freeze_to` to freeze all except the last two parameter groups:

仅在一个时期内，我们就得到了与<<chapter_intro>>中的训练相同的结果：还不错！我们可以将 `-2` 传递给 `freeze_to` 来冻结除最后两个参数组之外的所有参数组：

In [95]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(1e-2/(2.6**4),1e-2))

epoch,train_loss,valid_loss,accuracy,time
0,0.261510,0.198116,0.925120,01:17


Then we can unfreeze a bit more, and continue training:

然后我们可以再解冻一点，然后继续训练：

In [96]:
learn.freeze_to(-3)
learn.fit_one_cycle(1, slice(5e-3/(2.6**4),5e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.217656,0.182132,0.932080,01:35


And finally, the whole model!

In [ ]:
learn.unfreeze()
learn.fit_one_cycle(2, slice(1e-3/(2.6**4),1e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.172888,0.153770,0.943120,01:01
1,0.161492,0.155567,0.942640,00:57


We reached 94.3% accuracy, which was state-of-the-art performance just three years ago. By training another model on all the texts read backwards and averaging the predictions of those two models, we can even get to 95.1% accuracy, which was the state of the art introduced by the ULMFiT paper. It was only beaten a few months ago, by fine-tuning a much bigger model and using expensive data augmentation techniques (translating sentences in another language and back, using another model for translation).

Using a pretrained model let us build a fine-tuned language model that was pretty powerful, to either generate fake reviews or help classify them. This is exciting stuff, but it's good to remember that this technology can also be used for malign purposes.

我们的准确率达到了 94.3%，这在三年前还是最先进的。通过对所有向后阅读的文本训练另一个模型并对这两个模型的预测进行平均，我们甚至可以达到 95.1% 的准确率，这是 ULMFiT 论文介绍的最先进技术。几个月前，通过微调一个更大的模型并使用昂贵的数据增强技术（用另一种语言翻译句子并返回，使用另一种翻译模型），它才被击败。

使用预训练模型，我们可以构建一个非常强大的微调语言模型，可以生成虚假评论或帮助对它们进行分类。这是令人兴奋的事情，但最好记住这项技术也可以用于恶意目的。

## Disinformation and Language Models

Even simple algorithms based on rules, before the days of widely available deep learning language models, could be used to create fraudulent accounts and try to influence policymakers. Jeff Kao, now a computational journalist at ProPublica, analyzed the comments that were sent to the US Federal Communications Commission (FCC) regarding a 2017 proposal to repeal net neutrality. In his article ["More than a Million Pro-Repeal Net Neutrality Comments Were Likely Faked"](https://hackernoon.com/more-than-a-million-pro-repeal-net-neutrality-comments-were-likely-faked-e9f0e3ed36a6), he reports how he discovered a large cluster of comments opposing net neutrality that seemed to have been generated by some sort of Mad Libs-style mail merge. In <<disinformation>>, the fake comments have been helpfully color-coded by Kao to highlight their formulaic nature.

<img src="images/ethics/image16.png" width="700" id="disinformation" caption="Comments received by the FCC during the net neutrality debate">

Kao estimated that "less than 800,000 of the 22M+ comments… could be considered truly unique" and that "more than 99% of the truly unique comments were in favor of keeping net neutrality."

Given advances in language modeling that have occurred since 2017, such fraudulent campaigns could be nearly impossible to catch now.  You now have all the necessary tools at your disposal to create a compelling language model—that is, something that can generate context-appropriate, believable text. It won't necessarily be perfectly accurate or correct, but it will be plausible. Think about what this technology would mean when put together with the kinds of disinformation campaigns we have learned about in recent years. Take a look at the Reddit dialogue shown in <<ethics_reddit>>, where a language model based on OpenAI's GPT-2 algorithm is having a conversation with itself about whether the US government should cut defense spending.

<img src="images/ethics/image14.png" id="ethics_reddit" caption="An algorithm talking to itself on Reddit" alt="An algorithm talking to itself on Reddit" width="600">

In this case, it was explicitly said that an algorithm was used, but imagine what would happen if a bad actor decided to release such an algorithm across social networks. They could do it slowly and carefully, allowing the algorithm to gradually develop followers and trust over time. It would not take many resources to have literally millions of accounts doing this. In such a situation we could easily imagine getting to a point where the vast majority of discourse online was from bots, and nobody would have any idea that it was happening.

We are already starting to see examples of machine learning being used to generate identities. For example, <<katie_jones>> shows a LinkedIn profile for Katie Jones.

<img src="images/ethics/image15.jpeg" width="400" id="katie_jones" caption="Katie Jones's LinkedIn profile">

Katie Jones was connected on LinkedIn to several members of mainstream Washington think tanks. But she didn't exist. That image you see was auto-generated by a generative adversarial network, and somebody named Katie Jones has not, in fact, graduated from the Center for Strategic and International Studies.

Many people assume or hope that algorithms will come to our defense here—that we will develop classification algorithms that can automatically recognise autogenerated content. The problem, however, is that this will always be an arms race, in which better classification (or discriminator) algorithms can be used to create better generation algorithms.

## Conclusion

In this chapter we explored the last application covered out of the box by the fastai library: text. We saw two types of models: language models that can generate texts, and a classifier that determines if a review is positive or negative. To build a state-of-the art classifier, we used a pretrained language model, fine-tuned it to the corpus of our task, then used its body (the encoder) with a new head to do the classification.

Before we end this section, we'll take a look at how the fastai library can help you assemble your data for your specific problems.

## Questionnaire

1. What is "self-supervised learning"?
1. What is a "language model"?
1. Why is a language model considered self-supervised?
1. What are self-supervised models usually used for?
1. Why do we fine-tune language models?
1. What are the three steps to create a state-of-the-art text classifier?
1. How do the 50,000 unlabeled movie reviews help us create a better text classifier for the IMDb dataset?
1. What are the three steps to prepare your data for a language model?
1. What is "tokenization"? Why do we need it?
1. Name three different approaches to tokenization.
1. What is `xxbos`?
1. List four rules that fastai applies to text during tokenization.
1. Why are repeated characters replaced with a token showing the number of repetitions and the character that's repeated?
1. What is "numericalization"?
1. Why might there be words that are replaced with the "unknown word" token?
1. With a batch size of 64, the first row of the tensor representing the first batch contains the first 64 tokens for the dataset. What does the second row of that tensor contain? What does the first row of the second batch contain? (Careful—students often get this one wrong! Be sure to check your answer on the book's website.)
1. Why do we need padding for text classification? Why don't we need it for language modeling?
1. What does an embedding matrix for NLP contain? What is its shape?
1. What is "perplexity"?
1. Why do we have to pass the vocabulary of the language model to the classifier data block?
1. What is "gradual unfreezing"?
1. Why is text generation always likely to be ahead of automatic identification of machine-generated texts?


1. 什么是“自我监督学习”？
1. 什么是“语言模型”？
1. 为什么语言模型被认为是自我监督的？
1. 自监督模型通常用于什么？
1. 为什么要微调语言模型？
1. 创建最先进的文本分类器的三个步骤是什么？
1. 50,000 条未标记的电影评论如何帮助我们为 IMDb 数据集创建更好的文本分类器？
1. 为语言模型准备数据的三个步骤是什么？
1. 什么是“通证化”？为什么我们需要它？
1. 说出三种不同的标记化方法。
1. 什么是“xxbos”？
1. 列出 fastai 在标记化过程中应用于文本的四个规则。
1. 为什么重复的字符被替换为显示重复次数和重复字符的标记？
1. 什么是“数值化”？
1. 为什么有些单词会被“未知单词”标记替换？
1. 批量大小为 64 时，代表第一批的张量的第一行包含数据集的前 64 个标记。该张量的第二行包含什么？第二批的第一行包含什么？ （小心——学生们经常会弄错这一点！一定要在本书的网站上检查你的答案。）
1. 为什么文本分类需要padding？为什么我们不需要它来进行语言建模？
1. NLP 的嵌入矩阵包含什么？它的形状是什么？
1. 什么是“困惑”？
1. 为什么我们必须将语言模型的词汇传递给分类器数据块？
1. 什么是“逐步解冻”？
1. 为什么文本生成总是可能领先于机器生成文本的自动识别？

### Further Research

1. See what you can learn about language models and disinformation. What are the best language models today? Take a look at some of their outputs. Do you find them convincing? How could a bad actor best use such a model to create conflict and uncertainty?
1. Given the limitation that models are unlikely to be able to consistently recognize machine-generated texts, what other approaches may be needed to handle large-scale disinformation campaigns that leverage deep learning?

1. 看看你能学到什么关于语言模型和虚假信息的知识。当今最好的语言模型是什么？看看他们的一些输出。你觉得他们有说服力吗？坏人如何才能最好地利用这样的模型来制造冲突和不确定性？
1. 鉴于模型不太可能一致识别机器生成的文本的限制，可能需要哪些其他方法来处理利用深度学习的大规模虚假信息活动？